In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import json
import h5py
import glob

from SiPMStudio.analysis.dark import amp_dt, all_dts, cross_talk_frac, gain
from matplotlib.colors import LogNorm

from scipy.signal import find_peaks
from scipy.stats import linregress

In [2]:
file_list = glob.glob("t2/*room*.h5")

In [25]:
file_list
idx = 5

In [26]:
peaks_json = None
with open("ketek_room_reprocs_norm.json", "r") as json_file:
    peaks_json = json.load(json_file)

In [27]:
head, file_name = os.path.split(file_list[idx])
peaks = peaks_json["processes"]["normalize_charge"][file_name]["peak_locs"]
errors = peaks_json["processes"]["normalize_charge"][file_name]["peak_errors"]

In [28]:
gain(peaks, errors)

2271845.19091712

In [29]:
test_file = h5py.File(file_list[idx], "r")
bias = test_file["bias"][()]
dt = test_file["dt"][()]
timetags = test_file["timetag"][:]
blr_wf = test_file["/processed/channels/sipm/blr_wf"][:]
wf_len = test_file["/processed/channels/sipm/wf_len"][()]
adc_v = test_file["/processed/channels/sipm/adc_to_v"][()]
charges = test_file["/processed/channels/sipm/charge"][:]
norm_charges = test_file["/processed/channels/sipm/norm_charge"][:]

In [30]:
test_file.close()

In [31]:
wf_time = np.arange(0, dt*wf_len, dt)

In [32]:
def plot_waveforms(ax, time, waveforms, num=50, offset=0):
    for i in range(offset, num+offset):
        ax.plot(time, waveforms[i])
    ax.set_xlabel("Time (ns)")
    ax.set_ylabel("Amplitude (mV)")
    
def plot_charge(ax, charges, bins=1000, x_range=None):
    ax.hist(charges, bins=bins, range=x_range, histtype="stepfilled", alpha=0.15, color=sns.color_palette()[0])
    ax.hist(charges, bins=bins, range=x_range, histtype="step", color=sns.color_palette()[0])
    ax.set_xlabel("P.E.")
    ax.set_ylabel("Counts")
    
def plot_dts(ax, dts, bins=1000, x_range=None):
    ax.hist(dts, bins=bins, range=x_range, histtype="stepfilled", alpha=0.15, color=sns.color_palette()[0])
    n, bins, patches = ax.hist(dts, bins=bins, range=x_range, histtype="step", color=sns.color_palette()[0])
    ax.set_xlabel("Inter-times (ns)")
    ax.set_ylabel("Count")
    return n, bins
    
def plot_amp_dt(fig, ax, dts, amps, bins=[1000, 1000], x_range=[10, 5e3], y_range=[0, 600]):
    x_bins = np.logspace(np.log10(x_range[0]), np.log10(x_range[1]), bins[0])
    y_bins = np.linspace(y_range[0], y_range[1], bins[1])
    h, x, y, im = ax.hist2d(dts, amps, bins=[x_bins, y_bins], norm=LogNorm())
    ax.set_xlabel("Inter-times (ns)")
    ax.set_ylabel("Amplitude (mV)")
    ax.set_xscale("log")
    fig.colorbar(im, ax=ax)
    
def fit_exp(x, y):
    x_new = x[y > 0]
    y_new = y[y > 0]
    ln_y = np.log(y_new)
    slope, intercept, r, p, stderr = linregress(x_new, ln_y)
    scale = (1 / (1e-9*1e3))
    return abs(slope)*scale, stderr*scale, slope, intercept

In [33]:
fig, ax = plt.subplots()
plot_waveforms(ax, wf_time, blr_wf*adc_v*1e3)
plt.savefig(os.path.join(os.getcwd(), "plots", "waveforms", f"waveforms_{bias}.png"), dpi=600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
fig, ax = plt.subplots()
plot_charge(ax, charges)
ax.set_xlabel("Charge (C)")
ax.set_yscale("log")
plt.savefig(os.path.join(os.getcwd(), "plots", "pe_spectrums", f"charge_spec_{bias}.png"), dpi=600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
fig, ax = plt.subplots()
plot_charge(ax, norm_charges)
ax.set_yscale("log")
plt.savefig(os.path.join(os.getcwd(), "plots", "pe_spectrums", f"norm_spec_{bias}.png"), dpi=600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
wf_dts, wf_amps, wf_ids = amp_dt(timetags, blr_wf*adc_v*1e3, dt, height=100, distance=100, width=4)

 80%|████████  | 60442/75199 [00:09<00:02, 6383.21it/s]

In [36]:
fig, ax = plt.subplots()
plot_amp_dt(fig, ax, wf_dts, wf_amps, bins=[200, 1000], x_range=[10, 1e4], y_range=[0, 400])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
#wf_dts = all_dts(timetags, blr_wf*adc_v*1e3, dt, height=35, distance=100, width=4)

In [38]:
fig, ax = plt.subplots()
n, bins = plot_dts(ax, wf_dts, x_range=[0, 1e4], bins=150)
ax.set_yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
bin_centers = (bins[1:] + bins[:-1]) / 2
cut = (bin_centers < 6000) & (bin_centers > 1000)
rate, error, slope, intercept = fit_exp(bin_centers[cut], n[cut])

In [40]:
inter_times = np.linspace(0, 1e4, 100)
exp_func = np.exp(intercept + slope*inter_times)

In [41]:
fig, ax = plt.subplots()
n, bins = plot_dts(ax, wf_dts, x_range=[0, 1e4], bins=150)
ax.plot(inter_times, exp_func, color="red", alpha=0.5, label=rf"{round(rate, 1)} kHz $\pm$ {round(error, 2)}")
ax.legend()
ax.set_yscale("log")
plt.savefig(os.path.join(os.getcwd(), "plots", "dark_counts", f"dcr_{bias}.png"), dpi=600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
cross_talk, cross_error = cross_talk_frac(norm_charges)

In [ ]:
print(cross_talk, cross_error)